In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import cross_val_score

In [ ]:
df = pd.read_csv('data')
df

In [ ]:
df['car_ID'].nunique()

In [ ]:
df = pd.read_csv('data', index_col='car_ID')
df

In [ ]:
df.info()

In [ ]:
df.describe().T

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
df.nunique().sort_values(ascending=False)

In [ ]:
df.hist(grid=False, bins=8, figsize=(16, 10));

In [ ]:
plt.figure(figsize=(16,10))
sns.heatmap(df.corr(numeric_only=True), annot = True)

In [ ]:
df_copy = df.copy()

In [ ]:
num_cols = [col for col in df_copy.columns if pd.api.types.is_numeric_dtype(df_copy[col]) and col != 'price']
cat_cols = [col for col in df_copy.columns if col not in num_cols and col != 'price']

In [ ]:
len(num_cols), len(cat_cols), df_copy.shape[1]

In [ ]:
df_copy = df_copy[(np.abs(stats.zscore(df_copy[num_cols])) < 3).all(axis=1)]

In [ ]:
df.shape[0] - df_copy.shape[0]

In [ ]:
# sns.pairplot(df_copy)

In [ ]:
df_copy[cat_cols].nunique()

In [ ]:
df_copy[['brand', 'model']] = df_copy['CarName'].str.split(' ', n=1, expand=True)

In [ ]:
df_copy['model'] = df_copy['model'].str.lower().replace(' ', '')
df_copy['model'] = df_copy['model'].str.replace('(', '')
df_copy['model'] = df_copy['model'].str.replace(')', '')
df_copy = df_copy.drop(['CarName', 'model'], axis=1)

In [ ]:
df_copy['brand'].value_counts()

In [ ]:
df_copy

In [ ]:
cat_cols = [col for col in df_copy.columns if col not in num_cols and col != 'price' and col != 'CarName']
df_copy = pd.get_dummies(df_copy, columns=cat_cols, drop_first=True)
df_copy

In [ ]:
X = df_copy.drop('price', axis=1)
y = df_copy['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

In [ ]:
scaler = StandardScaler()
preprocessor = ColumnTransformer([('numeric', scaler, num_cols)], remainder='passthrough')

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

In [ ]:
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
r2_score(y_test, y_pred)

In [ ]:
mean_absolute_error(y_test, y_pred)

In [ ]:
mean_squared_error(y_test, y_pred)

In [ ]:
import pickle

pickle.dump(model, open("data", "wb"))